
# **Midterm Project: Bayesian Linear Model, Ridge, and Lasso Regression for MPG Prediction**

In this Jupyter Notebook, we will explore and apply Bayesian Linear Model, Ridge, and Lasso regression techniques to predict the miles per gallon (MPG) of cars based on various features. The objective is to employ these regression models with variable selection techniques using 5-fold cross-validation to optimize the Root Mean Squared Error (RMSE).

## Overview

The dataset used for this project contains information about different cars, including features such as cylinders, displacement, horsepower, weight, acceleration, model year, origin, and possibly others. The goal is to predict the MPG of cars using these features.

### Goals

1. Implement Bayesian Linear Model, Ridge, and Lasso regression models.
2. Utilize 5-fold cross-validation for model evaluation and hyperparameter tuning.
3. Achieve a smaller RMSE with Bayesian Ridge and Lasso compared to Bayesian Linear Model.
4. Explore techniques for reducing collinearity, such as removing redundant categorical features, to potentially improve prediction accuracy of Bayesian Ridge and Lasso models.

### Methodology

- **Data Loading and Preparation**: Load the MPG dataset and preprocess as necessary (handling missing values, encoding categorical variables).
  
- **Model Implementation**:
  - **Bayesian Linear Model**: Establish a baseline using Bayesian regression.
  - **Ridge Regression**: Apply Ridge regression with hyperparameter tuning using cross-validation.
  - **Lasso Regression**: Implement Lasso regression with cross-validation for feature selection.
  
- **Evaluation**: Compare the performance of Bayesian Ridge and Lasso models with the Bayesian Linear Model in terms of RMSE using 5-fold cross-validation.
  
- **Feature Selection**: Explore methods to reduce collinearity and improve model performance, particularly for Bayesian Ridge and Lasso models.


By the end of this notebook, we aim to have a clear understanding of how Bayesian Ridge and Lasso regression models perform in predicting MPG, compared to a Bayesian Linear Model. Additionally, we will explore the impact of feature selection techniques on model accuracy and provide insights into further improving predictions.

Let's dive into the implementation!

---

In [2]:
library(MASS)


gibbs_sampler_blr <- function(X, y, n_iter = 1000, burn_in = 100) {
  n <- nrow(X)
  p <- ncol(X)

  # Prior parameters
  beta_prior_mean <- rep(0, p)
  beta_prior_cov <- diag(p)
  sigma2_prior_shape <- 0.01
  sigma2_prior_scale <- 0.01

  # Storage for samples
  beta_samples <- matrix(0, nrow = n_iter, ncol = p)
  sigma2_samples <- numeric(n_iter)

  # Initial values
  beta <- rep(0, p)
  sigma2 <- 1

  for (iter in 1:n_iter) {
    # Sample beta | sigma2, y
    beta_cov <- solve(t(X) %*% X / sigma2 + solve(beta_prior_cov))
    beta_mean <- beta_cov %*% (t(X) %*% y / sigma2)
    beta <- MASS::mvrnorm(1, beta_mean, beta_cov)

    # Sample sigma2 | beta, y
    resid <- y - X %*% beta
    sigma2_shape <- sigma2_prior_shape + n / 2
    sigma2_scale <- sigma2_prior_scale + sum(resid^2) / 2
    sigma2 <- 1 / stats::rgamma(1, shape = sigma2_shape, rate = sigma2_scale)

    # Store samples
    if (iter > burn_in) {
      beta_samples[iter, ] <- beta
      sigma2_samples[iter] <- sigma2
    }
  }

  list(beta_samples = beta_samples[(burn_in + 1):n_iter, ],
       sigma2_samples = sigma2_samples[(burn_in + 1):n_iter])
}


# Data Preprocessing

in this first step, we will be using the nessecary libraries in order to load and preform accurate bayesian models in order get a better read of the data and see any simple observations

In [7]:
library(ggplot2)
data(mpg)


# Use all available features for prediction
data_mpg <- na.omit(mpg)
data_mpg$drv <- as.factor(data_mpg$drv)
data_mpg$fl <- as.factor(data_mpg$fl)
data_mpg$class <- as.factor(data_mpg$class)

X <- model.matrix(hwy ~ ., data = data_mpg)[, -1]
y <- data_mpg$hwy
n <- length(y)

#Defining Bayesian Models using nessecary Functions

In [8]:
gibbs_sampler_brr <- function(X, y, n_iter = 1000, burn_in = 100, lambda = 0.1) {
  n <- nrow(X)
  p <- ncol(X)

  # Prior parameters
  beta_prior_mean <- rep(0, p)
  beta_prior_cov <- diag(p) / lambda
  sigma2_prior_shape <- 0.01
  sigma2_prior_scale <- 0.01

  # Storage for samples
  beta_samples <- matrix(0, nrow = n_iter, ncol = p)
  sigma2_samples <- numeric(n_iter)

  # Initial values
  beta <- rep(0, p)
  sigma2 <- 1

  for (iter in 1:n_iter) {
    # Sample beta | sigma2, y
    beta_cov <- solve(t(X) %*% X / sigma2 + solve(beta_prior_cov))
    beta_mean <- beta_cov %*% (t(X) %*% y / sigma2)
    beta <- MASS::mvrnorm(1, beta_mean, beta_cov)

    # Sample sigma2 | beta, y
    resid <- y - X %*% beta
    sigma2_shape <- sigma2_prior_shape + n / 2
    sigma2_scale <- sigma2_prior_scale + sum(resid^2) / 2
    sigma2 <- 1 / stats::rgamma(1, shape = sigma2_shape, rate = sigma2_scale)

    # Store samples
    beta_samples[iter, ] <- beta
    sigma2_samples[iter] <- sigma2
  }

  list(beta_samples = beta_samples[(burn_in + 1):n_iter, ],
       sigma2_samples = sigma2_samples[(burn_in + 1):n_iter])
}

# Example usage:
 brr_samples <- gibbs_sampler_brr(X, y)
 beta_samples_brr <- brr_samples$beta_samples
 sigma2_samples_brr <- brr_samples$sigma2_samples


In [9]:
gibbs_sampler_lasso <- function(X, y, n_iter = 1000, burn_in = 100, lambda = 1) {
  n <- nrow(X)
  p <- ncol(X)

  # Storage for samples
  beta_samples <- matrix(0, nrow = n_iter, ncol = p)
  sigma2_samples <- numeric(n_iter)
  lambda_samples <- matrix(0, nrow = n_iter, ncol = p)

  # Initial values
  beta <- rep(0, p)
  sigma2 <- 1
  lambda2 <- rep(lambda^2, p)

  for (iter in 1:n_iter) {
    # Sample beta | sigma2, lambda, y
    beta_cov <- solve(t(X) %*% X / sigma2 + diag(1 / lambda2))
    beta_mean <- beta_cov %*% (t(X) %*% y / sigma2)
    beta <- MASS::mvrnorm(1, beta_mean, beta_cov)

    # Sample sigma2 | beta, y
    resid <- y - X %*% beta
    sigma2_shape <- 0.01 + n / 2
    sigma2_scale <- 0.01 + sum(resid^2) / 2
    sigma2 <- 1 / stats::rgamma(1, shape = sigma2_shape, rate = sigma2_scale)

    # Sample lambda2 | beta
    lambda2 <- 1 / stats::rgamma(p, shape = 1, rate = beta^2 / (2 * sigma2) + 1 / lambda^2)

    # Store samples
    beta_samples[iter, ] <- beta
    sigma2_samples[iter] <- sigma2
    lambda_samples[iter, ] <- sqrt(lambda2)
  }

  list(beta_samples = beta_samples[(burn_in + 1):n_iter, ],
       sigma2_samples = sigma2_samples[(burn_in + 1):n_iter],
       lambda_samples = lambda_samples[(burn_in + 1):n_iter, ])
}

# Function to tune lambda
tune_lambda <- function(X, y, lambdas, top_n = 4, n_iter = 1000, burn_in = 100) {
  best_lambda <- NULL
  min_diff <- Inf

  for (lambda in lambdas) {
    lasso_samples <- gibbs_sampler_lasso(X, y, n_iter, burn_in, lambda)
    beta_means <- colMeans(lasso_samples$beta_samples)
    significant <- abs(apply(lasso_samples$beta_samples, 2, quantile, probs = 0.025)) > 0 &
                   abs(apply(lasso_samples$beta_samples, 2, quantile, probs = 0.975)) > 0
    nonzero_count <- sum(significant)
    diff <- abs(nonzero_count - top_n)

    if (diff < min_diff) {
      min_diff <- diff
      best_lambda <- lambda
    }
  }

  best_lambda
}

# Example usage:
 lambdas <- seq(0.1, 1, by = 0.1)
 best_lambda <- tune_lambda(X, y, lambdas)
lasso_samples <- gibbs_sampler_lasso(X, y, lambda = best_lambda)
beta_samples_lasso <- lasso_samples$beta_samples
sigma2_samples_lasso <- lasso_samples$sigma2_samples


In [10]:
install.packages("caret")
library(caret)

# Function to calculate RMSE
calculate_rmse <- function(y_true, y_pred) {
  sqrt(mean((y_true - y_pred)^2))
}

# Function to perform cross-validation and compute RMSE for each model
cross_validate_models <- function(X, y, lambdas = seq(0.1, 1, by = 0.1)) {
  set.seed(123)
  folds <- createFolds(y, k = 5, list = TRUE, returnTrain = TRUE)

  blr_rmse <- numeric(5)
  brr_rmse <- numeric(5)
  lasso_rmse <- numeric(5)

  for (i in 1:5) {
    train_indices <- folds[[i]]
    test_indices <- setdiff(1:nrow(X), train_indices)

    X_train <- X[train_indices, ]
    y_train <- y[train_indices]
    X_test <- X[test_indices, ]
    y_test <- y[test_indices]

    # Bayesian Linear Regression
    blr_samples <- gibbs_sampler_blr(X_train, y_train)
    beta_blr <- colMeans(blr_samples$beta_samples)
    lower_blr <- apply(blr_samples$beta_samples, 2, quantile, probs = 0.025)
    upper_blr <- apply(blr_samples$beta_samples, 2, quantile, probs = 0.975)
    significant_blr <- which(sign(lower_blr) == sign(upper_blr))
    y_pred_blr <- X_test[, significant_blr] %*% beta_blr[significant_blr]
    blr_rmse[i] <- calculate_rmse(y_test, y_pred_blr)

    # Bayesian Ridge Regression
    brr_samples <- gibbs_sampler_brr(X_train, y_train)
    beta_brr <- colMeans(brr_samples$beta_samples)
    lower_brr <- apply(brr_samples$beta_samples, 2, quantile, probs = 0.025)
    upper_brr <- apply(brr_samples$beta_samples, 2, quantile, probs = 0.975)
    significant_brr <- which(sign(lower_brr) == sign(upper_brr))
    y_pred_brr <- X_test[, significant_brr] %*% beta_brr[significant_brr]
    brr_rmse[i] <- calculate_rmse(y_test, y_pred_brr)

    # Bayesian Lasso
    best_lambda <- 1  # Change to tune_lambda(X_train, y_train, lambdas) for automated lambda selection
    lasso_samples <- gibbs_sampler_lasso(X_train, y_train, lambda = best_lambda)
    beta_lasso <- colMeans(lasso_samples$beta_samples)
    lower_lasso <- apply(lasso_samples$beta_samples, 2, quantile, probs = 0.025)
    upper_lasso <- apply(lasso_samples$beta_samples, 2, quantile, probs = 0.975)
    significant_lasso <- which(sign(lower_lasso) == sign(upper_lasso))
    y_pred_lasso <- X_test[, significant_lasso] %*% beta_lasso[significant_lasso]
    lasso_rmse[i] <- calculate_rmse(y_test, y_pred_lasso)
  }

  blr_rmse_mean <- mean(blr_rmse)
  brr_rmse_mean <- mean(brr_rmse)
  lasso_rmse_mean <- mean(lasso_rmse)

  results <- data.frame(
    Model = c("Bayesian Linear Regression", "Bayesian Ridge Regression", "Bayesian Lasso"),
    RMSE = c(blr_rmse_mean, brr_rmse_mean, lasso_rmse_mean)
  )

  return(results)
}

# Example usage:
# Replace X and y with your actual data
lambdas <- seq(0.1, 2, by = 0.1)
 results <- cross_validate_models(X, y, lambdas)
print(results)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



                       Model     RMSE
1 Bayesian Linear Regression 1.619512
2  Bayesian Ridge Regression 2.571378
3             Bayesian Lasso 2.341974


In [11]:
library(ggplot2)
library(MASS)  # For mvrnorm function (multivariate normal sampling)

# Load the mpg dataset
data(mpg)
# Assuming you want to predict highway mileage (hwy) using other variables

# Prepare the predictor matrix X and response vector y
X <- model.matrix(hwy ~ . - 1, data = mpg)  # Exclude intercept (column of 1s)
y <- mpg$hwy

# Function to simulate Bayesian samples for Linear Regression
gibbs_sampler_blr <- function(X, y, n_iter = 1000, burn_in = 100) {
  n <- nrow(X)
  p <- ncol(X)

  # Prior parameters
  beta_prior_mean <- rep(0, p)
  beta_prior_cov <- diag(1, p)
  sigma2_prior_shape <- 0.01
  sigma2_prior_scale <- 0.01

  # Storage for samples
  beta_samples <- matrix(0, nrow = n_iter, ncol = p)
  sigma2_samples <- numeric(n_iter)

  # Initial values
  beta <- rep(0, p)
  sigma2 <- 1

  for (iter in 1:n_iter) {
    # Sample beta | sigma2, y
    beta_cov <- solve(t(X) %*% X / sigma2 + solve(beta_prior_cov))
    beta_mean <- beta_cov %*% (t(X) %*% y / sigma2)
    beta <- mvrnorm(1, beta_mean, beta_cov)

    # Sample sigma2 | beta, y
    resid <- y - X %*% beta
    sigma2_shape <- sigma2_prior_shape + n / 2
    sigma2_scale <- sigma2_prior_scale + sum(resid^2) / 2
    sigma2 <- 1 / rgamma(1, shape = sigma2_shape, rate = sigma2_scale)

    # Store samples
    beta_samples[iter, ] <- beta
    sigma2_samples[iter] <- sigma2
  }

  list(beta_samples = beta_samples[(burn_in + 1):n_iter, ],
       sigma2_samples = sigma2_samples[(burn_in + 1):n_iter])
}

# Simulate Bayesian samples
blr_samples <- gibbs_sampler_blr(X, y)

# Compute summary statistics
blr_summary <- data.frame(
  Mean = colMeans(blr_samples$beta_samples),
  Median = apply(blr_samples$beta_samples, 2, median),
  Lower = apply(blr_samples$beta_samples, 2, quantile, probs = 0.025),
  Upper = apply(blr_samples$beta_samples, 2, quantile, probs = 0.975)
)

# Determine significance based on quantiles
blr_summary$Significant <- with(blr_summary, sign(Lower) == sign(Upper))

# Assign row names from original predictor matrix X
rownames(blr_summary) <- colnames(X)

# Print the summary
print(blr_summary)





                                     Mean       Median        Lower
manufactureraudi             1.1386371479  1.150901090 -0.145397465
manufacturerchevrolet        0.1640565588  0.154410733 -0.972616671
manufacturerdodge           -0.5518427022 -0.539272192 -1.763753119
manufacturerford            -0.4432806753 -0.468149025 -1.577565578
manufacturerhonda           -0.0606804274 -0.040794484 -1.624635050
manufacturerhyundai          0.0396482352  0.030920059 -1.236685479
manufacturerjeep            -0.3957971453 -0.405018713 -1.865875457
manufacturerland rover       0.4833079779  0.483454943 -0.976330094
manufacturerlincoln         -0.0201941875 -0.040332720 -1.602153688
manufacturermercury         -0.1112935031 -0.121220418 -1.657220620
manufacturernissan          -0.5525080582 -0.596203413 -1.754048640
manufacturerpontiac          0.5991705331  0.637724588 -0.944965379
manufacturersubaru           0.3840716386  0.372373045 -0.968481392
manufacturertoyota          -0.7067197639 -0.688

In [12]:
# Assuming you have already defined gibbs_sampler_brr function and loaded the data

# Example: Bayesian Ridge Regression
brr_samples <- gibbs_sampler_brr(X, y)

# Compute summary statistics
brr_summary <- data.frame(
  Mean = colMeans(brr_samples$beta_samples),
  Median = apply(brr_samples$beta_samples, 2, median),
  Lower = apply(brr_samples$beta_samples, 2, quantile, probs = 0.025),
  Upper = apply(brr_samples$beta_samples, 2, quantile, probs = 0.975)
)

# Determine significance based on quantiles
brr_summary$Significant <- with(brr_summary, sign(Lower) == sign(Upper))

# Assign row names from original predictor matrix X
rownames(brr_summary) <- colnames(X)

# Print the summary
print(brr_summary)

# Example: Using significant variables for further analysis
significant_vars <- colnames(X)[brr_summary$Significant == TRUE]

# Example: Fit a linear model using significant variables
lm_model <- lm(y ~ X[, significant_vars] - 1)

# Example: Print summary of the linear model
print(summary(lm_model))


                                    Mean       Median        Lower        Upper
manufactureraudi             1.308440402  1.305921187 -2.379770180  5.341824862
manufacturerchevrolet        0.096591022  0.079484390 -3.274670827  3.573476808
manufacturerdodge           -0.541940886 -0.465305676 -4.416875647  3.041270217
manufacturerford            -0.585700388 -0.593506262 -3.840431484  2.928842560
manufacturerhonda           -0.154902107 -0.162480811 -4.721926036  4.596334625
manufacturerhyundai         -0.149332928 -0.166213324 -4.083327886  3.734699286
manufacturerjeep            -0.322616206 -0.273927073 -4.853178503  4.130926795
manufacturerland rover       0.643310995  0.531663750 -3.743088871  5.270471505
manufacturerlincoln          0.020926866  0.138440305 -4.705641203  4.532256973
manufacturermercury          0.093186387  0.092845611 -4.245471076  4.603954473
manufacturernissan          -0.670676117 -0.703750775 -4.094137496  2.798670091
manufacturerpontiac          0.454881029

In [13]:
# Define a sequence of lambda values to tune
lambdas <- seq(0.1, 5, by = 0.1)

# Tune lambda using the provided function
best_lambda <- tune_lambda(X, y, lambdas)

# Run the Gibbs sampler for Lasso with the best lambda
lasso_samples <- gibbs_sampler_lasso(X, y, lambda = best_lambda)

# Compute summary statistics
lasso_summary <- data.frame(
  Mean = colMeans(lasso_samples$beta_samples),
  Median = apply(lasso_samples$beta_samples, 2, median),
  Lower = apply(lasso_samples$beta_samples, 2, quantile, probs = 0.025),
  Upper = apply(lasso_samples$beta_samples, 2, quantile, probs = 0.975)
)

# Determine significance based on quantiles
lasso_summary$Significant <- with(lasso_summary, sign(Lower) == sign(Upper))

# Assign row names from original predictor matrix X
rownames(lasso_summary) <- colnames(X)

# Print the summary
print(lasso_summary)


                                   Mean       Median         Lower       Upper
manufactureraudi            -0.25540091 -0.447475552 -1.519226e+01 14.83297220
manufacturerchevrolet       -0.58203925 -0.455841481 -1.439768e+01 13.64268818
manufacturerdodge           -1.35579803 -1.375067214 -1.528053e+01 12.71971686
manufacturerford            -1.04646340 -1.472294621 -1.391126e+01 12.97177195
manufacturerhonda           -0.77846678 -0.464640357 -2.875827e+01 25.50546047
manufacturerhyundai         -0.15909779 -0.259517355 -1.824210e+01 16.05019841
manufacturerjeep            -0.09755033 -0.445350898 -2.538902e+01 24.19854295
manufacturerland rover       0.91382678  0.316629216 -2.691668e+01 34.51041755
manufacturerlincoln          0.22673963  0.167966039 -2.939956e+01 30.85839549
manufacturermercury         -0.58877051 -0.608872517 -3.348486e+01 26.37254272
manufacturernissan          -1.09892492 -0.828230767 -1.561943e+01 14.20703958
manufacturerpontiac         -0.07879267 -0.007675437

In [14]:
summary(lm(y ~ X[, lasso_summary$Significant == TRUE]-1))



Call:
lm(formula = y ~ X[, lasso_summary$Significant == TRUE] - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.9378 -1.1012  0.0008  1.2320  4.3248 

Coefficients:
                                                    Estimate Std. Error t value
X[, lasso_summary$Significant == TRUE]cyl            0.05861    0.06235   0.940
X[, lasso_summary$Significant == TRUE]transauto(s5)  1.08210    0.95993   1.127
X[, lasso_summary$Significant == TRUE]cty            1.33798    0.01969  67.965
X[, lasso_summary$Significant == TRUE]fle           -0.24957    0.80063  -0.312
X[, lasso_summary$Significant == TRUE]flp            1.64398    0.57684   2.850
X[, lasso_summary$Significant == TRUE]flr            0.23978    0.54746   0.438
                                                    Pr(>|t|)    
X[, lasso_summary$Significant == TRUE]cyl            0.34823    
X[, lasso_summary$Significant == TRUE]transauto(s5)  0.26081    
X[, lasso_summary$Significant == TRUE]cty            < 2e-16 ***
X[,

In [15]:
summary(lm(y ~ X[, brr_summary$Significant == TRUE]-1))



Call:
lm(formula = y ~ X[, brr_summary$Significant == TRUE] - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.1740 -1.1248  0.0026  1.2076  4.2784 

Coefficients:
                                                    Estimate Std. Error t value
X[, brr_summary$Significant == TRUE]year          -0.0007036  0.0009521  -0.739
X[, brr_summary$Significant == TRUE]cyl            0.1347461  0.1204589   1.119
X[, brr_summary$Significant == TRUE]transauto(s5)  1.0583256  0.9614264   1.101
X[, brr_summary$Significant == TRUE]cty            1.3715577  0.0495210  27.696
X[, brr_summary$Significant == TRUE]fle            0.2458309  1.0448431   0.235
X[, brr_summary$Significant == TRUE]flp            2.0413210  0.7889931   2.587
X[, brr_summary$Significant == TRUE]flr            0.6378365  0.7684117   0.830
                                                  Pr(>|t|)    
X[, brr_summary$Significant == TRUE]year            0.4607    
X[, brr_summary$Significant == TRUE]cyl             0.2645 

In [16]:
summary(lm(y ~ X[, blr_summary$Significant == TRUE]-1))



Call:
lm(formula = y ~ X[, blr_summary$Significant == TRUE] - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6356 -0.7129  0.0231  0.6965  3.2807 

Coefficients:
                                                    Estimate Std. Error t value
X[, blr_summary$Significant == TRUE]year           0.0026858  0.0003054   8.795
X[, blr_summary$Significant == TRUE]transauto(l5)  0.7250792  0.2243789   3.231
X[, blr_summary$Significant == TRUE]transauto(s5)  1.1057387  0.7054891   1.567
X[, blr_summary$Significant == TRUE]drvf           0.9217179  0.2894034   3.185
X[, blr_summary$Significant == TRUE]drvr           1.1946571  0.3037979   3.932
X[, blr_summary$Significant == TRUE]cty            1.0837484  0.0291125  37.226
X[, blr_summary$Significant == TRUE]fle           -1.1741781  0.4670856  -2.514
X[, blr_summary$Significant == TRUE]flp            0.6349920  0.2107897   3.012
X[, blr_summary$Significant == TRUE]classpickup   -2.6603458  0.3806505  -6.989
X[, blr_summary$Significa